In [1]:
import sys
sys.path.append("/root/HSE_diploma/")
sys.path.append("/root/HSE_diploma/evaluator/")

In [2]:
!nvidia-smi
# t2c_concode_220428_v38_plusoneepoch

Fri May 26 12:18:25 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:41:00.0 Off |                  N/A |
| 30%   25C    P8    15W / 350W |      1MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from utils.utils import load_model_tokenizer_from_pretrained, draw_metrics_compare_with_glue
from ParamsIterator import ParamsIterator
from EvaluateTestSet import EvaluateTestSet
import pandas as pd
from transformers import GenerationConfig

from tqdm import tqdm_notebook
import json

/root/HSE_diploma/prompter/templates/

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)


In [4]:
# default_model = "yahma/llama-7b-hf"
# experiment_name = "t2c_concode_220428_v38_plusoneepoch"

experiment_name = "/root/experiments/t2c_concode_220428_v38_plustwoepoch_test_copy/"
default_model = json.load(open("/root/experiments/t2c_concode_220428_v38_plustwoepoch_test_copy/experiment_config.json", "r"))['default_model']
params_iteration = {"temperature": [1.0],
                    "max_new_tokens": [300]#None, 20, 30, 35, 40, 50, 60, 70, 80, 90, 100] + [45, 47, 49, 51, 53, 55]
                   }


In [5]:
default_model

'yahma/llama-7b-hf'

In [6]:
experiment_name, params_iteration

('/root/experiments/t2c_concode_220428_v38_plustwoepoch_test_copy/',
 {'temperature': [1.0], 'max_new_tokens': [300]})

In [7]:
tokenizer, model = load_model_tokenizer_from_pretrained(default_model = default_model, 
                                                        experiment_name = experiment_name
                                                       )

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
def verbose_model_tokenizer(model, tokenizer):
    print("tokenizer.pad_token_id", tokenizer.pad_token_id)
    print("tokenizer.eos_token_id", tokenizer.eos_token_id)
    print("tokenizer.bos_token_id", tokenizer.bos_token_id)
    print("tokenizer.eos_token_id", tokenizer.eos_token_id)

    print("model.config.pad_token_id", model.config.pad_token_id)
    print("model.config.eos_token_id", model.config.eos_token_id)
    print("model.config.bos_token_id", model.config.bos_token_id)
    print("model.config.eos_token_id", model.config.eos_token_id)
    model.print_trainable_parameters()
    
verbose_model_tokenizer(model, tokenizer)

# tokenizer.pad_token_id 0
# tokenizer.eos_token_id 2
# tokenizer.bos_token_id 1
# tokenizer.eos_token_id 2
# model.config.pad_token_id 0
# model.config.eos_token_id 2
# model.config.bos_token_id 1
# model.config.eos_token_id 2
# trainable params: 0 || all params: 6746804224 || trainable%: 0.0


tokenizer.pad_token_id 0
tokenizer.eos_token_id 2
tokenizer.bos_token_id 1
tokenizer.eos_token_id 2
model.config.pad_token_id 0
model.config.eos_token_id 2
model.config.bos_token_id 1
model.config.eos_token_id 2
trainable params: 0 || all params: 7543721984 || trainable%: 0.0


In [9]:
from tqdm import tqdm_notebook

def get_metric_res(model, tokenizer, params_iteration, experiment_name):
    res = []
    for generation_config_dict in tqdm_notebook(ParamsIterator(params_iteration=params_iteration)):
        evaluator = EvaluateTestSet(generation_config = GenerationConfig(**generation_config_dict
                                                                        ),
                                    
                                    #fn_test_data = "temp/t2c_answers.json",
                                    #fn_etalon = "temp/answers.json"
                                   )

        metric_res = evaluator.evaluate(model=model, 
                                        tokenizer=tokenizer                                        
                                       )
        for key, val in generation_config_dict.items():
            assert key not in metric_res
            metric_res[key] = val

        metric_res['experiment_name'] = experiment_name
        print(generation_config_dict, metric_res)

        res.append(metric_res)
    
    return res

In [10]:
tokenizer.pad_token_id = model.config.pad_token_id = 0

In [11]:
model.eval();

In [12]:
# import torch
# from transformers import GenerationConfig

# from datetime import datetime
# import json
# import numpy as np
# import math
# import tqdm

# from T2CEvaluator import T2CEvaluator

# from prompter import generate_test_prompt, get_response
# # prompter = Prompter()

# # def generate_test_prompt(data_point):
# #     #assert 'output' not in data_point or data_point['output']==''
# #     if "input" in data_point and data_point["input"]:
# #         return prompter.generate_prompt(instruction = data_point["instruction"],
# #                                         input = data_point["input"],
# #                                         #label = ''#data_point["output"]
# #                                        )
# #     else:
# #         return prompter.generate_prompt(instruction = data_point["instruction"],
# #                                         #input = None,
# #                                         #label = ''#data_point["output"]
# #                                        )
   
# # def generate_test_prompt(data_point, train = False):
# #     # To decrease expectations of results :)
# #     assert train == False
# #     # sorry about the formatting disaster gotta move fast
# #     if data_point["input"]:
# #         return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
# # ### Instruction:
# # {data_point["instruction"]}

# # ### Input:
# # {data_point["input"]}

# # ### Response:
# # {data_point["output"] if train else ''}"""
# #     else:
# #         return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
# # ### Instruction:
# # {data_point["instruction"]}

# # ### Response:
# # {data_point["output"] if train else ''}"""


# class EvaluateTestSet:
#     def __init__(self, 
#                  generation_config = GenerationConfig(max_new_tokens = 128), 
#                  fn_test_data = "../data/t2c_answers.json",
#                  fn_etalon = "/root/data/answers.json",
#                  batch_size = 10,
#                  verbose = False
#                 ):
#         self.generation_config = generation_config
        
#         self.fn_test_data = fn_test_data
#         self.fn_etalon = fn_etalon
        
#         self.batch_size = batch_size
#         self.verbose = verbose
       
#     def preprocess(self, s):
#         #ToDo rewrite it using Promt Template
#         s = get_response(s)
#         #s = s.split('### Response:\n')[-1]
#         s = s.replace('\n', '  ')
#         s = s.replace('<unk>', " ")
#         s = ' '.join(s.split(' ')[:100])
#         while '  ' in s:
#             s = s.replace('  ', ' ')

#         if len(s) > 0 and s[0] == ' ':
#             s = s[1:]
        
#         if self.verbose:
#             print(s)
        
#         return s

#     def clean_results(self, res_list):
#         predict_list = []
#         for s in tqdm.tqdm(res_list):
#             predict_list.append(self.preprocess(s))
#         return predict_list
    
#     def get_raw_results(self, model, tokenizer, prompts):
#         batch_size = self.batch_size
#         generation_config = self.generation_config
        
#         res_list = []
#         n = math.ceil(len(prompts)/batch_size)
#         n = 1
#         for ind in tqdm.tqdm(range(n)):
#             current_prompts = prompts[ind*batch_size: (ind+1)*batch_size]
#             if self.verbose:
#                 print(ind * batch_size, (ind+1)*batch_size, len(current_prompts))

#             tokenized_inputs = tokenizer(list(current_prompts), 
#                                          padding=True, 
#                                          truncation=True, 
#                                          return_tensors="pt"
#                                         ).to('cuda')



#             with torch.no_grad():
#                 full_output = model.generate(
#                     **tokenized_inputs,
#                     generation_config=generation_config
#                 )

#             res_list.extend(tokenizer.batch_decode(full_output, skip_special_tokens=False))
#             self.res_list = res_list
#         return res_list
    
#     def save_results(self, predict_list):
#         output_filename = str(datetime.now()).split('.')[0].replace(' ', '-').replace(':', '_')+'.txt'
#         fn_output = "/root/results/%s"%output_filename
        
#         res = '\n'.join([i if i!='' else '-' for i in predict_list])
#         open(fn_output, "w+", encoding='utf-8').write(res)
#         return fn_output
    
#     def evaluate(self, model, tokenizer):
#         model.eval()
#         assert model.training == False

#         lst = json.load(open(self.fn_test_data, 'rb'))
#         inputs = lst# [lst[0]]
#         # instruction = 'Combine the question and answer into an image caption as succinctly as possible. Be sure to include the phrase "a photo of". Do not draw false conclusions.'
#         # inputs = ['Is this a baseball game? yes', 'Is this a baseball game? no']
#         prompts = [generate_test_prompt(inp) for inp in inputs]
#         prompts = np.array(prompts)
        
#         res_list = self.get_raw_results(model = model, 
#                                         tokenizer = tokenizer,
#                                         prompts = prompts)
        
#         model.train()
#         assert model.training == True
        
#         predict_list = self.clean_results(res_list)
        
#         self.fn_output = self.save_results(predict_list)
        
#         t2c_evaluator = T2CEvaluator()
#         metric_res = t2c_evaluator.calculate_metrics(fn_answers = self.fn_etalon, 
#                                                      fn_predictions = self.fn_output
#                                                      )
#         return metric_res

In [13]:
experiment_name

'/root/experiments/t2c_concode_220428_v38_plustwoepoch_test_copy/'

In [14]:
evaluator = None
res = []
for generation_config_dict in tqdm_notebook(ParamsIterator(params_iteration=params_iteration)):
    evaluator = EvaluateTestSet(generation_config = GenerationConfig(**generation_config_dict
                                                                    ),

                                #fn_test_data = "temp/t2c_answers.json",
                                #fn_etalon = "temp/answers.json"
                               )

    metric_res = evaluator.evaluate(model=model, 
                                    tokenizer=tokenizer                                        
                                   )
    for key, val in generation_config_dict.items():
        assert key not in metric_res
        metric_res[key] = val

    metric_res['experiment_name'] = experiment_name
    print(generation_config_dict, metric_res)

    res.append(metric_res)


/tmp/ipykernel_5059/272965761.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for generation_config_dict in tqdm_notebook(ParamsIterator(params_iteration=params_iteration)):


  0%|          | 0/1 [00:00<?, ?it/s]


  0%|                                                    | 0/10 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.

100%|███████████████████████████████████████████| 10/10 [09:33<00:00, 57.31s/it]

100%|██████████████████████████████████████| 100/100 [00:00<00:00, 44002.35it/s]

{'max_new_tokens': 300, 'temperature': 1.0} {'EM': 0.0, 'BLEU': 0.03065082934202577, 'brevity_penalty': 1.0, 'ratio': 1.3741956241956241, 'translation_length': 4271, 'reference_length': 3108, 'precisions_0': 0.07279962546816479, 'precisions_1': 0.037152444870565675, 'precisions_2': 0.022347740667976426, 'precisions_3': 0.01460221550855992, 'max_new_tokens': 300, 'temperature': 1.0, 'experiment_name': '/root/experiments/t2c_concode_220428_v38_plustwoepoch_test_copy/'}


{'max_new_tokens': 300, 'temperature': 1.0} {'EM': 0.0, 'BLEU': 0.03065082934202577, 'brevity_penalty': 1.0, 'ratio': 1.3741956241956241, 'translation_length': 4271, 'reference_length': 3108, 'precisions_0': 0.07279962546816479, 'precisions_1': 0.037152444870565675, 'precisions_2': 0.022347740667976426, 'precisions_3': 0.01460221550855992, 'max_new_tokens': 300, 'temperature': 1.0, 'experiment_name': '/root/experiments/t2c_concode_220428_v38_plustwoepoch/'}from tqdm import tqdm_notebook

In [14]:
# {'max_new_tokens': 300, 'temperature': 1.0} {'EM': 0.0, 'BLEU': 0.03065082934202577, 'brevity_penalty': 1.0, 'ratio': 1.3741956241956241, 'translation_length': 4271, 'reference_length': 3108, 'precisions_0': 0.07279962546816479, 'precisions_1': 0.037152444870565675, 'precisions_2': 0.022347740667976426, 'precisions_3': 0.01460221550855992, 'max_new_tokens': 300, 'temperature': 1.0, 'experiment_name': '/root/experiments/t2c_concode_220428_v38_plustwoepoch/'}from tqdm import tqdm_notebook

/tmp/ipykernel_5005/1950585116.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for generation_config_dict in tqdm_notebook(ParamsIterator(params_iteration=params_iteration)):


  0%|          | 0/1 [00:00<?, ?it/s]


  0%|                                                    | 0/10 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.

100%|███████████████████████████████████████████| 10/10 [09:18<00:00, 55.88s/it]

100%|██████████████████████████████████████| 100/100 [00:00<00:00, 45095.19it/s]

{'max_new_tokens': 300, 'temperature': 1.0} {'EM': 0.0, 'BLEU': 0.03065082934202577, 'brevity_penalty': 1.0, 'ratio': 1.3741956241956241, 'translation_length': 4271, 'reference_length': 3108, 'precisions_0': 0.07279962546816479, 'precisions_1': 0.037152444870565675, 'precisions_2': 0.022347740667976426, 'precisions_3': 0.01460221550855992, 'max_new_tokens': 300, 'temperature': 1.0, 'experiment_name': '/root/experiments/t2c_concode_220428_v38_plustwoepoch/'}


/tmp/ipykernel_4967/1950585116.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for generation_config_dict in tqdm_notebook(ParamsIterator(params_iteration=params_iteration)):


  0%|          | 0/1 [00:00<?, ?it/s]


  0%|                                                    | 0/10 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.

100%|███████████████████████████████████████████| 10/10 [04:09<00:00, 24.91s/it]

100%|██████████████████████████████████████| 100/100 [00:00<00:00, 64369.31it/s]

{'max_new_tokens': 300, 'temperature': 1.0} {'EM': 0.0, 'BLEU': 0.3165820184021644, 'brevity_penalty': 0.7845045406807699, 'ratio': 0.8046975546975547, 'translation_length': 2501, 'reference_length': 3108, 'precisions_0': 0.6342925659472423, 'precisions_1': 0.46253122398001667, 'precisions_2': 0.34317984361424847, 'precisions_3': 0.2633969118982743, 'max_new_tokens': 300, 'temperature': 1.0, 'experiment_name': '/root/experiments/t2c_concode_220428_v38_plusoneepoch/'}


0

/tmp/ipykernel_3716/1950585116.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for generation_config_dict in tqdm_notebook(ParamsIterator(params_iteration=params_iteration)):


  0%|          | 0/1 [00:00<?, ?it/s]


  0%|                                                    | 0/10 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.

100%|███████████████████████████████████████████| 10/10 [03:36<00:00, 21.65s/it]

100%|██████████████████████████████████████| 100/100 [00:00<00:00, 73365.47it/s]

{'max_new_tokens': 300, 'temperature': 1.0} {'EM': 0.0, 'BLEU': 0.27863233355465167, 'brevity_penalty': 0.6950607457027734, 'ratio': 0.7332689832689833, 'translation_length': 2279, 'reference_length': 3108, 'precisions_0': 0.6416666666666667, 'precisions_1': 0.4596330275229358, 'precisions_2': 0.3382988947621336, 'precisions_3': 0.2588294651866801, 'max_new_tokens': 300, 'temperature': 1.0, 'experiment_name': '/root/experiments/t2c_concode_220428_v37/'}


/tmp/ipykernel_3586/1950585116.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for generation_config_dict in tqdm_notebook(ParamsIterator(params_iteration=params_iteration)):


  0%|          | 0/1 [00:00<?, ?it/s]


  0%|                                                    | 0/10 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.

100%|███████████████████████████████████████████| 10/10 [03:28<00:00, 20.84s/it]

100%|██████████████████████████████████████| 100/100 [00:00<00:00, 72178.70it/s]

{'max_new_tokens': 300, 'temperature': 1.0} {'EM': 0.0, 'BLEU': 0.27863233355465167, 'brevity_penalty': 0.6950607457027734, 'ratio': 0.7332689832689833, 'translation_length': 2279, 'reference_length': 3108, 'precisions_0': 0.6416666666666667, 'precisions_1': 0.4596330275229358, 'precisions_2': 0.3382988947621336, 'precisions_3': 0.2588294651866801, 'max_new_tokens': 300, 'temperature': 1.0, 'experiment_name': '/root/experiments/t2c_concode_220428_v37/'}


In [12]:
evaluator

AttributeError: 'EvaluateTestSet' object has no attribute 'res_list'

In [21]:
pred_list = [get_response(s) for s in evaluator.res_list]

'String function ( ) { return baseZNode + "srini_string" + IDENTITIES_ZNODE_NAME ; }</s><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk>'

In [ ]:
evaluator.res_list

In [14]:
res3 = get_metric_res(model=model,
                     tokenizer=tokenizer,
                     params_iteration=params_iteration,
                     experiment_name=experiment_name
                    )
res3

/tmp/ipykernel_3481/3839643825.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for generation_config_dict in tqdm_notebook(ParamsIterator(params_iteration=params_iteration)):


  0%|          | 0/1 [00:00<?, ?it/s]


  0%|                                                     | 0/1 [00:08<?, ?it/s]

KeyboardInterrupt



In [15]:
from prompter import prompter, get_response

In [16]:
s = """### Response:
TernaryBool function ( ) { return canBeInstantiated ( ) ; }</s><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk>
===
<unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Generate java code
interpret a short as its binary form 

### Input:
 PlaceHolder placeHolder 

 long asLong 
 int toInt 
 String toBinaryString 
 String toBinaryString 
 String toBinaryString 
 byte[] fromInt 
 byte[] fromLong

### Response:
short function ( final short arg0 ) { return ( short ) ( arg0 & 0xFFFF ) ; }</s><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk>"""

In [19]:
o = EvaluateTestSet()
o.preprocess(s)

### Response:
TernaryBool function ( ) { return canBeInstantiated ( ) ; }</s><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk>
===
<unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><un

'### Response: TernaryBool function ( ) { return canBeInstantiated ( ) ; }</s> === '

In [17]:
get_response(s)

### Response:
TernaryBool function ( ) { return canBeInstantiated ( ) ; }</s><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk>
===
<unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><un

'short function ( final short arg0 ) { return ( short ) ( arg0 & 0xFFFF ) ; }</s><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk>'

/tmp/ipykernel_1969/3839643825.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for generation_config_dict in tqdm_notebook(ParamsIterator(params_iteration=params_iteration)):


  0%|          | 0/1 [00:00<?, ?it/s]


  0%|                                                    | 0/10 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.

100%|███████████████████████████████████████████| 10/10 [02:15<00:00, 13.54s/it]

100%|██████████████████████████████████████| 100/100 [00:00<00:00, 34314.85it/s]

{'max_new_tokens': 200, 'temperature': 1.0} {'EM': 0.0, 'BLEU': 0.0006187833664486746, 'brevity_penalty': 0.30601594809335797, 'ratio': 0.45785070785070786, 'translation_length': 1423, 'reference_length': 3108, 'precisions_0': 0.03651685393258427, 'precisions_1': 0.0007552870090634441, 'precisions_2': 0.0007710100231303007, 'precisions_3': 0.0007861635220125787, 'max_new_tokens': 200, 'temperature': 1.0, 'experiment_name': '/root/experiments/t2c_concode_220428_v34/'}


[{'EM': 0.0,
  'BLEU': 0.0006187833664486746,
  'brevity_penalty': 0.30601594809335797,
  'ratio': 0.45785070785070786,
  'translation_length': 1423,
  'reference_length': 3108,
  'precisions_0': 0.03651685393258427,
  'precisions_1': 0.0007552870090634441,
  'precisions_2': 0.0007710100231303007,
  'precisions_3': 0.0007861635220125787,
  'max_new_tokens': 200,
  'temperature': 1.0,
  'experiment_name': '/root/experiments/t2c_concode_220428_v34/'}]